In [5]:
import json

# Load JSON from file
with open('raw_response.json', 'r') as f:
    data = json.load(f)

# Extract lighthouse result
lh = data.get("lighthouseResult", {})
audits = lh.get("audits", {})

# Basic Scores & Metrics
output = {
    "performance_score": round(lh.get("categories", {}).get("performance", {}).get("score", 0) * 100),
    "seo_score": round(lh.get("categories", {}).get("seo", {}).get("score", 0) * 100),
    "first_contentful_paint": audits.get("first-contentful-paint", {}).get("displayValue"),
    "largest_contentful_paint": audits.get("largest-contentful-paint", {}).get("displayValue"),
    "total_blocking_time": audits.get("total-blocking-time", {}).get("displayValue"),
    "speed_index": audits.get("speed-index", {}).get("displayValue"),
    "cumulative_layout_shift": audits.get("cumulative-layout-shift", {}).get("displayValue"),
    "time_to_interactive": audits.get("interactive", {}).get("displayValue"),
    "created_at": lh.get("fetchTime"),
    "updated_at": lh.get("fetchTime"),
}

# Fix Suggestions
fixes = []
for audit_id, audit in audits.items():
    if audit.get("score") is not None and audit.get("score") < 1:
        details = audit.get("details", {})
        items = details.get("items", [])
        url = None

        # Safely try to extract URL if present
        if isinstance(items, list) and len(items) > 0 and isinstance(items[0], dict):
            url = items[0].get("url")

        fix = {
            "id": audit.get("id"),
            "title": audit.get("title"),
            "description": audit.get("description"),
            "display_value": audit.get("displayValue", ""),
            "url": url
        }
        fixes.append(fix)

# Output the summary
print("🔍 Key Metrics:")
for k, v in output.items():
    print(f"{k}: {v}")

print("\n🛠️  Fix Recommendations:")
for fix in fixes:
    print(f"- {fix['title']}")
    print(f"  ↪ Description: {fix['description']}")
    if fix["display_value"]:
        print(f"  ↪ Detail: {fix['display_value']}")
    if fix["url"]:
        print(f"  ↪ URL: {fix['url']}")
    print()


🔍 Key Metrics:
performance_score: 93
seo_score: 100
first_contentful_paint: 0.9 s
largest_contentful_paint: 1.5 s
total_blocking_time: 20 ms
speed_index: 1.6 s
cumulative_layout_shift: 0.021
time_to_interactive: 1.5 s
created_at: 2025-04-10T07:14:06.807Z
updated_at: 2025-04-10T07:14:06.807Z

🛠️  Fix Recommendations:
- Defer offscreen images
  ↪ Description: Consider lazy-loading offscreen and hidden images after all critical resources have finished loading to lower time to interactive. [Learn how to defer offscreen images](https://developer.chrome.com/docs/lighthouse/performance/offscreen-images/).
  ↪ Detail: Potential savings of 131 KiB
  ↪ URL: https://aiattorney.com.pk/assets/Case%20diary-BTRwfGEM.png

- Third parties
  ↪ Description: Third party code can significantly impact load performance. [Reduce and defer loading of third party code](https://developers.google.com/web/fundamentals/performance/optimizing-content-efficiency/loading-third-party-javascript/) to prioritize your pag

In [ ]:
const fs = require('fs');
const ReportGenerator = require('lighthouse/report/generator/report-generator');

const json = JSON.parse(fs.readFileSync('your-lighthouse-data.json', 'utf8'));
const html = ReportGenerator.generateReport(json, 'html');

fs.writeFileSync('report.html', html);
